# 统计日最大负荷最高的10天数据所属的季节

In [ ]:
import pandas as pd

# data有下面几列数据：地域、季节、日最大负荷
# 想统计日最大负荷最高的10天/20属于各个季节的个数
def count_seasons(df):
    # 找到日最大负荷最大的10个数据所在的日期
    top_dates = df.nlargest(10, "日最大负荷")["季节"]
    # 计算这些日期属于春季、夏季、秋季和冬季的个数
    spring_count = sum(top_dates.str.endswith("春季"))
    summer_count = sum(top_dates.str.endswith("夏季"))
    autumn_count = sum(top_dates.str.endswith("秋季"))
    winter_count = sum(top_dates.str.endswith("冬季"))
    # 返回结果
    return [spring_count, summer_count, autumn_count, winter_count]

# 按城市分组，并应用函数
result = data.groupby("地域").apply(count_seasons)
# 将结果转换为数据框，并添加列名
result = pd.DataFrame(result.tolist(), index=result.index, columns=["春季个数", "夏季个数", "秋季个数", "冬季个数"])

# 找气象负荷占比差值最大的温度区间

In [ ]:
# filtered_data有下面几列数据：地域、温度区间、该温度区间气象负荷占比均值、上一个温度区间气象负荷占比、温度区间气象负荷占比差值
# 找到温度区间气象负荷占比差值最大的温度区间

def find_max_temp_range(group):
    # 找到温度区间气象负荷占比差值最大的那个温度区间
    max_temp_range = group.loc[group['温度区间气象负荷占比差值'].idxmax(), '温度区间']
    # 将这个温度区间添加到数据框中作为新的一列
    group['最大温度区间'] = max_temp_range
    return group

data_temp = filtered_data.groupby('地域').apply(find_max_temp_range)

# 积温累积效应的计算

In [ ]:
# data_jw有下面几列数据："地域","日期",'日最高气温',"临界温度","温度是否超过临界值"
# 想要实现积温累积效应的计算：（日最高气温-临界温度）*连续超过临界温度的天数
# 计算积温累积效应
def accum_temperature(df):
    # 按照地域进行分组
    grouped = df.groupby('地域')
    # 对于每个地域，计算温度连续超过气温临界值的天数，并计算积温累积效应
    for name, group in grouped:
        # 初始化连续超过天数为0
        group['连续超过天数'] = 0
        # 计算积温累积效应
        group['积温累积效应'] = (group['日最高气温'] - group['临界温度']) * group['连续超过天数']
        # 对于每个日期，判断温度是否超过气温临界值
        for i in range(0, len(group)-1):
            if i==0:
                if group.loc[group.index[i], '温度是否超过临界值'] == 1:
                    # 如果温度超过气温临界值，则将连续超过天数加1
                    group.loc[group.index[i], '连续超过天数'] = 1
                else:
                    # 如果温度低于气温临界值，则将连续超过天数重置为0
                    group.loc[group.index[i], '连续超过天数'] = 0
            else: 
                if group.loc[group.index[i], '温度是否超过临界值'] == 1:
                    # 如果温度超过气温临界值，则将连续超过天数加1
                    group.loc[group.index[i], '连续超过天数'] = group.loc[group.index[i-1], '连续超过天数'] + 1
                else:
                    # 如果温度低于气温临界值，则将连续超过天数重置为0
                    group.loc[group.index[i], '连续超过天数'] = 0
            # 计算积温累积效应
            group.loc[group.index[i], '积温累积效应'] = (group.loc[group.index[i], '日最高气温'] - group.loc[group.index[i], '临界温度']) * group.loc[group.index[i], '连续超过天数']
        # 将计算结果保存到原数据框中
        df.loc[group.index, '积温累积效应'] = group['积温累积效应']
        df.loc[group.index, '连续超过天数'] = group['连续超过天数']
    df["积温累积效应"]=df["积温累积效应"].apply(lambda x:round(x,2))
    return df

data_jw=data[["地域","日期",'日最高气温',"临界温度","温度是否超过临界值"]]
data_jw=accum_temperature(data_jw)

# 找到气象负荷高于2.5倍均值的异常高值，并使用均值填充 

In [ ]:
# data有地域、气象负荷两列数据
# 处理城市整体气象负荷的异常高值
def abnormal_handling(df):
    # 计算每个城市的均值和标准差
    grouped = df.groupby("地域")
#     for elec in ["气象负荷",'城镇居民生活负荷-气象负荷','乡村居民生活负荷-气象负荷','一般工商业负荷-气象负荷']:
    for elec in ["气象负荷"]:
        means = grouped.mean()[f"{elec}"]
        stds = grouped.std()[f"{elec}"]
        # 处理每个城市的异常高值数据
        for city in means.index:
            # 找到该城市的异常高值数据
            high_values = df.loc[(df["地域"] == city) & (df[f"{elec}"] > means[city] + 2.5 * stds[city]), f"{elec}"]
            while not high_values.empty:
                # 用其余数据的均值来填充异常高值数据
                data.loc[(df["地域"] == city) & (df[f"{elec}"] > means[city] + 2.5 * stds[city]), f"{elec}"] = means[city]
                # 重新计算该城市的均值和标准差
                means[city] = df.loc[df["地域"] == city, f"{elec}"].mean()
                stds[city] = df.loc[df["地域"] == city, f"{elec}"].std()
                # 找到该城市的新的异常高值数据
                high_values = df.loc[(df["地域"] == city) & (df[f"{elec}"] > means[city] + 2.5 * stds[city]), f"{elec}"]
    return df


data=abnormal_handling(data)

# 不连续日期的数据展示

In [ ]:
# 数据data有三列数据：城市、日期、日最大负荷，日期是分为两段，
# 想要在图上不间断日期的展示日最大负荷的数据情况
# 假设数据框名为results
grouped_results = data.groupby('城市')

# 遍历每个城市的数据子集
for city, data_ in grouped_results:
    # 创建一个新的图表
    plt.figure(dpi=120)
    data_.drop_duplicates(subset=["日期"],keep="last",inplace=True)
    
    
    data_2022 = data_[data_['日期'] <= '2022-08-31']
    data_2023 = data_[data_['日期'] >= '2023-05-27']
    data_2022["日期"]=data_2022["日期"].apply(lambda x:str(x)[:10])
    data_2023["日期"]=data_2023["日期"].apply(lambda x:str(x)[:10])
    
    data_["日期"]=data_["日期"].apply(lambda x:str(x)[:10])
    
    
    # 绘制折线图
    plt.plot(data_2022['日期'], data_2022['日最大负荷'], 'g-', data_2023['日期'], data_2023['日最大负荷'],'r--')
    
    la= data_['日期'].iloc[::20]
    plt.xticks(ticks=list(range(0,len(data_['日期']),20)),        #设置要显示的x轴刻度，若指定空列表则去掉x轴刻度, 
    labels=la,    #设置x轴刻度显示的文字，要与ticks对应   
    fontsize=9,        #设置刻度字体大小
    rotation=0,        #设置刻度文字旋转角度
    ha='center', va='center')        #刻度文字对齐方式，当rotation_mode为’anchor'时，对齐方式决定了文字旋转的中心。ha也可以写成horizontalalignment，va也可以写成verticalalignment。)

    # 设置图表标题和轴标签
    plt.title(city + "预测的最大负荷变动情况")
    plt.xlabel("时间")
    plt.ylabel("最大负荷")
    # 显示图表
    #test111
    plt.show()